In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

print("🔍 Loading TODAY's intraday data...")

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🔍 Loading TODAY's intraday data...


In [2]:
# Tyr's holdings from screenshot
HOLDINGS = ['UUUU', 'USAR', 'NVTS', 'ASTS', 'SIDU']

# Get 1-minute data for today
today_data = {}
for ticker in HOLDINGS:
    try:
        stock = yf.Ticker(ticker)
        # Get 1-day of 1-minute bars
        hist = stock.history(period='1d', interval='1m')
        if not hist.empty:
            today_data[ticker] = hist
            print(f"✓ {ticker}: {len(hist)} bars")
    except Exception as e:
        print(f"✗ {ticker}: {e}")

print(f"\nLoaded {len(today_data)} tickers")

✓ UUUU: 311 bars
✓ USAR: 300 bars
✓ NVTS: 315 bars
✓ ASTS: 321 bars
✓ SIDU: 314 bars

Loaded 5 tickers


In [3]:
# Analyze each ticker's intraday pattern
print("="*70)
print("📊 INTRADAY ANALYSIS - JAN 6, 2026")
print("="*70)

for ticker, df in today_data.items():
    if df.empty:
        continue
        
    open_price = df['Open'].iloc[0]
    close_price = df['Close'].iloc[-1]
    high = df['High'].max()
    low = df['Low'].min()
    
    # Find the low point time
    low_idx = df['Low'].idxmin()
    low_time = low_idx.strftime('%H:%M') if hasattr(low_idx, 'strftime') else 'Unknown'
    
    # Calculate key metrics
    open_to_close = ((close_price - open_price) / open_price) * 100
    low_to_close = ((close_price - low) / low) * 100
    
    # Check for V-shape: low in morning, recovery by close
    morning_bars = df.iloc[:60]  # First hour
    morning_low = morning_bars['Low'].min()
    
    is_v_shape = (low == morning_low) and (low_to_close > 2)
    
    print(f"\n{ticker}")
    print("-"*70)
    print(f"Open:  ${open_price:.2f}")
    print(f"Low:   ${low:.2f} at {low_time}")
    print(f"High:  ${high:.2f}")
    print(f"Close: ${close_price:.2f}")
    print(f"")
    print(f"Open → Close: {open_to_close:+.2f}%")
    print(f"Low → Close:  {low_to_close:+.2f}%")
    print(f"")
    print(f"V-SHAPE? {'✓ YES' if is_v_shape else '✗ NO'}")
    
    if is_v_shape:
        print(f"  → Bottomed early, recovered {low_to_close:.1f}% from low")
        print(f"  → MISSED OPPORTUNITY if you sold the dip")

📊 INTRADAY ANALYSIS - JAN 6, 2026

UUUU
----------------------------------------------------------------------
Open:  $18.80
Low:   $17.72 at 13:06
High:  $19.02
Close: $18.43

Open → Close: -1.94%
Low → Close:  +4.03%

V-SHAPE? ✗ NO

USAR
----------------------------------------------------------------------
Open:  $16.24
Low:   $16.10 at 09:30
High:  $18.16
Close: $17.58

Open → Close: +8.22%
Low → Close:  +9.16%

V-SHAPE? ✓ YES
  → Bottomed early, recovered 9.2% from low
  → MISSED OPPORTUNITY if you sold the dip

NVTS
----------------------------------------------------------------------
Open:  $9.04
Low:   $9.01 at 09:30
High:  $10.21
Close: $10.13

Open → Close: +12.07%
Low → Close:  +12.44%

V-SHAPE? ✓ YES
  → Bottomed early, recovered 12.4% from low
  → MISSED OPPORTUNITY if you sold the dip

ASTS
----------------------------------------------------------------------
Open:  $91.00
Low:   $90.19 at 09:31
High:  $97.85
Close: $97.32

Open → Close: +6.95%
Low → Close:  +7.91%

V-S

In [4]:
# After-hours movement
print("\n" + "="*70)
print("🌙 AFTER-HOURS MOVEMENT")
print("="*70)

for ticker in HOLDINGS:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        regular_close = info.get('regularMarketPrice', 0)
        post_market = info.get('postMarketPrice', 0)
        
        if post_market and regular_close:
            ah_change = ((post_market - regular_close) / regular_close) * 100
            print(f"\n{ticker}")
            print(f"  Close: ${regular_close:.2f}")
            print(f"  AH:    ${post_market:.2f} ({ah_change:+.2f}%)")
    except:
        pass


🌙 AFTER-HOURS MOVEMENT

UUUU
  Close: $18.34
  AH:    $18.40 (+0.31%)

USAR
  Close: $17.52
  AH:    $18.10 (+3.31%)

NVTS
  Close: $10.19
  AH:    $10.21 (+0.19%)

ASTS
  Close: $97.49
  AH:    $96.55 (-0.96%)

SIDU
  Close: $4.94
  AH:    $5.39 (+9.11%)


In [5]:
# THE VERDICT
print("\n" + "="*70)
print("🐺 BROKKR VERDICT: WHAT SHOULD TYR DO?")
print("="*70)

print("""
If you're seeing V-shapes TODAY:

1. DO NOT panic sell intraday dips (you just did that and it recovered)
2. The "bleeding" you saw was likely the morning fade
3. If it V'd back, that's NOT a selling signal - it's strength

RULES GOING FORWARD:
- If you bought Day 0 (yesterday), you HOLD through Day 1
- Don't sell the open dip - that's the trap
- 10:30 AM is decision time: still red? Consider exit. Green? Hold.

NEXT STEPS:
1. Log what ACTUALLY happened today (use the daily tracker)
2. Did you panic sell? Log it. Learn from it.
3. Tomorrow: WATCH but don't trade. Observe patterns.
4. Paper trade for 3 days before real money moves.

The V-shape IS a pattern. But you need to EXPECT it, not react to it.
""")


🐺 BROKKR VERDICT: WHAT SHOULD TYR DO?

If you're seeing V-shapes TODAY:

1. DO NOT panic sell intraday dips (you just did that and it recovered)
2. The "bleeding" you saw was likely the morning fade
3. If it V'd back, that's NOT a selling signal - it's strength

RULES GOING FORWARD:
- If you bought Day 0 (yesterday), you HOLD through Day 1
- Don't sell the open dip - that's the trap
- 10:30 AM is decision time: still red? Consider exit. Green? Hold.

NEXT STEPS:
1. Log what ACTUALLY happened today (use the daily tracker)
2. Did you panic sell? Log it. Learn from it.
3. Tomorrow: WATCH but don't trade. Observe patterns.
4. Paper trade for 3 days before real money moves.

The V-shape IS a pattern. But you need to EXPECT it, not react to it.

